In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
from tqdm import tqdm

In [2]:
df = pd.read_csv('UB_clear.csv')

In [3]:
df['iid_str'] = df['iid'].apply(str)

In [4]:
df_pv = df[df['type']=='pv']

In [5]:
df_pv

,uid,iid,cid,type,timestamp,date,iid_str
0,1,2268318,2520377,pv,1511544070,2017-11-25,2268318
1,1,2333346,2520771,pv,1511561733,2017-11-25,2333346
2,1,2576651,149192,pv,1511572885,2017-11-25,2576651
3,1,3830808,4181361,pv,1511593493,2017-11-25,3830808
4,1,4365585,2520377,pv,1511596146,2017-11-25,4365585
...,...,...,...,...,...,...,...
100095226,999999,4797808,11120,pv,1512293403,2017-12-03,4797808
100095227,999999,4613472,4602841,pv,1512293766,2017-12-03,4613472
100095228,999999,3647364,2304296,pv,1512293792,2017-12-03,3647364
100095229,999999,1903801,2304296,pv,1512293827,2017-12-03,1903801


In [6]:
pd.set_option('display.float_format',lambda x : '%.f' % x)
df_pv.describe()

,uid,iid,cid,timestamp
count,89660688,89660688,89660688,89660688
mean,507221,2579576,2699025,1511961190
std,294136,1488052,1462244,230146
min,1,1,80,1511539200
25%,252584,1295012,1320293,1511761969
50%,504487,2580198,2671397,1511964585
75%,761656,3861799,4145813,1512179058
max,1018011,5163070,5162429,1512316799


In [7]:
dates = ['2017-11-25'] #, '2017-11-26', '2017-11-27', '2017-11-28', '2017-11-29', '2017-11-30', '2017-12-01' , '2017-12-02', '2017-12-03'

In [8]:
for date in dates:
    df_pv_day = df_pv[df_pv['date']==date]
    df_pv_day_grby_uid = df_pv_day.groupby('uid')
    iid_corpus = []
    not_exist_uid = []
    for _uid in tqdm(range(1018012)):
        try:
            iid_corpus.append(df_pv_day_grby_uid.get_group(_uid).iloc[:, 6].tolist())
        except:
            not_exist_uid.append(_uid)
    with open('./iid_corpus/' + 'iid_corpus_' + date + '.pkl', 'wb') as file:
        pickle.dump(iid_corpus, file)

100%|██████████| 1018012/1018012 [03:12<00:00, 5292.65it/s]


In [9]:
dates

['2017-11-25']

In [10]:
iid_corpus = []
for date in dates:
    with open('./iid_corpus/' + 'iid_corpus_' + date + '.pkl', 'rb') as file:
        iid_corpus_date = pickle.load(file)
        print(date + ' len: ' + str(len(iid_corpus_date)))
        iid_corpus += iid_corpus_date

2017-11-25 len: 686953


In [11]:
len(iid_corpus)

686953

---

In [13]:
import multiprocessing
from gensim.models import Word2Vec

In [14]:
starttime = datetime.datetime.now()

In [15]:
model = Word2Vec(iid_corpus, size=128, window=5, min_count=1, workers=multiprocessing.cpu_count())

In [16]:
endtime = datetime.datetime.now()
print(endtime - starttime)

0:04:19.875826


In [17]:
model.save('iid_1days.model')